In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Fetch NIFTY50 historical data from 01/01/2024 to 07/03/2025
nifty50 = yf.download('^NSEI', start='2024-01-01', end='2025-03-07')

# Calculate daily percentage returns and daily volatility (21-day rolling)
nifty50['Returns'] = nifty50['Close'].pct_change() * 100  # Daily percentage returns
nifty50['Volatility'] = nifty50['Returns'].rolling(window=21).std() * np.sqrt(252)

# Drop NaN values and reset the index
nifty50 = nifty50.dropna().reset_index()

# Display the first few rows of the processed data
print(nifty50.head())

# Save the analyzed data to a CSV file
nifty50.to_csv('NIFTY50_Analysis.csv', index=False)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price        Date         Close          High           Low          Open  \
Ticker                    ^NSEI         ^NSEI         ^NSEI         ^NSEI   
0      2024-02-01  21697.449219  21832.949219  21658.750000  21780.650391   
1      2024-02-02  21853.800781  22126.800781  21805.550781  21812.750000   
2      2024-02-05  21771.699219  21964.300781  21726.949219  21921.050781   
3      2024-02-06  21929.400391  21951.400391  21737.550781  21825.199219   
4      2024-02-07  21930.500000  22053.300781  21860.150391  22045.050781   

Price   Volume   Returns Volatility  
Ticker   ^NSEI                       
0       332500 -0.130030  15.489715  
1       442800  0.720599  15.632753  
2       440800 -0.375686  15.486488  
3       371000  0.724340  15.521154  
4       346300  0.005014  15.508718  


In [3]:
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px

# Load the analyzed data from the CSV file
nifty50 = pd.read_csv('NIFTY50_Analysis.csv')

# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1('NIFTY50 Performance Dashboard'),
    html.Div([
        html.H2('Daily % Price Return'),
        dcc.Graph(
            figure=px.line(nifty50, x='Date', y='Returns', title='NIFTY50 Daily % Price Return',
                           color_discrete_sequence=px.colors.qualitative.Set1)
        )
    ]),
])

# Run the app on a different port
if __name__ == '__main__':
    app.run_server(debug=True, port=8070)

# Prerequisites and Setup Instructions
# 1. Install required libraries: pip install pandas dash plotly
# 2. Ensure the 'NIFTY50_Analysis.csv' file is in the same directory as this script.
# 3. Compatible with Jupyter, VS Code, or any Python IDE.

In [4]:
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px

# Load the analyzed data from the CSV file
nifty50 = pd.read_csv('NIFTY50_Analysis.csv')

# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1('NIFTY50 Performance Dashboard'),
    html.Div([
        html.H2('Daily Volatility (21-Day Rolling)'),
        dcc.Graph(
            figure=px.line(nifty50, x='Date', y='Volatility', title='NIFTY50 Daily Volatility (21-Day Rolling)',
                           color_discrete_sequence=px.colors.qualitative.Bold)
        )
    ]),
])

# Run the app on a different port to avoid conflicts
if __name__ == '__main__':
    app.run_server(debug=False, port=8090)

In [5]:
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
from fpdf import FPDF
from datetime import datetime
import socket
import threading
import matplotlib.pyplot as plt

# Load the analyzed data from the CSV file
nifty50 = pd.read_csv('NIFTY50_Analysis.csv')
nifty50['Date'] = pd.to_datetime(nifty50['Date'])  # Ensure 'Date' is in datetime format

# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1('NIFTY50 Performance Dashboard'),
    html.Div([
        html.H2('Daily Returns'),
        dcc.Graph(
            figure=px.line(nifty50, x='Date', y='Returns', title='NIFTY50 Daily Returns',
                           color_discrete_sequence=px.colors.qualitative.Bold)
        )
    ]),
    html.Div([
        html.H2('Daily Volatility (21-Day Rolling)'),
        dcc.Graph(
            figure=px.line(nifty50, x='Date', y='Volatility', title='NIFTY50 Daily Volatility (21-Day Rolling)',
                           color_discrete_sequence=px.colors.qualitative.Bold)
        )
    ]),
])

# Function to find an available port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('localhost', 0))
        return s.getsockname()[1]

# Function to run the Dash app in a separate thread
def run_dash_app():
    app.run_server(debug=False, port=find_free_port())

# Start the Dash app in a separate thread
threading.Thread(target=run_dash_app).start()

# Generate a PDF report using FPDF
class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'NIFTY50 Analysis', 0, 1, 'C')
        self.set_font('Arial', '', 10)
        self.cell(0, 10, f'{datetime.today().strftime("%d/%b/%Y")}', 0, 0, 'R')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Create a PDF instance
pdf = PDFReport()
pdf.add_page()

# Cover Page
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'NIFTY50 Analysis', 0, 1, 'C')
pdf.set_font('Arial', '', 12)
pdf.cell(0, 10, 'Time Period: 01/Jan/2024 to 07/Mar/2025', 0, 1, 'C')
pdf.ln(10)

pdf.multi_cell(0, 10, 'What is NIFTY50?\n\nThe NIFTY50 is a benchmark Indian stock market index that represents the weighted average of 50 of the largest Indian companies listed on the National Stock Exchange (NSE). The index is widely used by investors as a measure of market performance and economic health. It covers various sectors, including financials, IT, energy, and consumer goods, providing a comprehensive overview of the Indian equity market.\n\nAuthor: Harshit Marwah')

# Generate and save combined chart as an image
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(nifty50['Date'], nifty50['Returns'], label='Daily Returns', color='blue')
plt.title('NIFTY50 Daily Returns')
plt.xlabel('Date')
plt.ylabel('Returns')
plt.xticks(rotation=45)

plt.subplot(2, 1, 2)
plt.plot(nifty50['Date'], nifty50['Volatility'], label='Volatility', color='red')
plt.title('NIFTY50 Daily Volatility (21-Day Rolling)')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('combined_charts.png')
plt.close()

# Add Combined Charts section with image to PDF
pdf.add_page()
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, 'Daily Returns and Volatility', 0, 1)
pdf.set_font('Arial', '', 12)
pdf.multi_cell(0, 10, '''The combined chart below illustrates both the daily returns and the 21-day rolling volatility of the NIFTY50 index over the selected period. Analyzing these metrics together helps in understanding market performance and risk levels.''')
pdf.image('combined_charts.png', x=10, y=30, w=190)

# Export PDF
pdf.output('NIFTY50_Complete_Report.pdf')

''